In [1]:
# Import Necessary Libraries
import os
import csv
import numpy as np
from skimage import io, color, filters
from skimage.feature import graycomatrix, graycoprops, hog, local_binary_pattern

c:\Users\pepe-\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# Define image directories
directories = {
    'glioma_tumor': 'Data base/glioma_tumor',
    'meningioma_tumor': 'Data base/meningioma_tumor',
    'pituitary_tumor': 'Data base/pituitary_tumor',
    'no_tumor': 'Data base/no_tumor'
}

In [3]:
# Create a dictionary to map categories to numbers
map_categories = {
    'glioma_tumor': 0,
    'meningioma_tumor': 1,
    'pituitary_tumor': 2,
    'no_tumor': 3
}

In [4]:
# Create the file .csv
with open('Data_comp.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write column headers
    writer.writerow(["Image", "Contrast", "Dissimilarity", "Homogeneity", "ASM", "Energy", "Correlation", "Entropy", 
                     "Gabor", "Gabor_0", "Gabor_45", "Gabor_90", "Gabor_135", "HOG", "LBP", "Category"])

# Iterate on each directory
    for category, directories in directories.items():
        for filename in os.listdir(directories):
            if filename.endswith(".jpg"):
                # Load Image
                image = io.imread(os.path.join(directories, filename))

                # Convert image to grayscale
                image_gray = color.rgb2gray(image)

                # Quantize the image in grayscale at 8 levels
                image_quan = np.floor(image_gray * 8).astype(np.uint8)

                # Calculate the gray level co-occurrence matrix.
                glcm = graycomatrix(image_quan, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)

                # Calculate Haralick's features
                contrast = graycoprops(glcm, 'contrast')[0, 0]
                dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
                homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
                ASM = graycoprops(glcm, 'ASM')[0, 0]
                energy = graycoprops(glcm, 'energy')[0, 0]
                correlation = graycoprops(glcm, 'correlation')[0, 0]

                # Calculate entropy
                entropy = -np.sum(glcm * np.log2(glcm + np.finfo(float).eps))

                # Category number
                categoria_num = map_categories[category]

                # Calculate Gabor's features
                gabor_real_filter, gabor_imaginary_filter = filters.gabor(image_gray, frequency=0.6)
                gabor = (gabor_real_filter ** 2 + gabor_imaginary_filter ** 2).mean()

                # Calculate the Gabor features for several orientations
                gabor_0 = filters.gabor(image_gray, frequency=0.6, theta=0)[0].mean()
                gabor_45 = filters.gabor(image_gray, frequency=0.6, theta=np.pi / 4)[0].mean()
                gabor_90 = filters.gabor(image_gray, frequency=0.6, theta=np.pi / 2)[0].mean()
                gabor_135 = filters.gabor(image_gray, frequency=0.6, theta=3 * np.pi / 4)[0].mean()

                # Calculate the features of HOG (Histograms of Oriented Gradients)
                hog_features = hog(image_gray, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), block_norm='L2-Hys')
                hog_features_mean = hog_features.mean()

                # Calculate LBP features (Local Binary Patterns)
                lbp = local_binary_pattern(image_gray, 8, 1, method="uniform")
                lbp_hist, _ = np.histogram(lbp, 8)
                lbp_hist = np.array(lbp_hist, dtype=float)
                lbp_prob = np.divide(lbp_hist, np.sum(lbp_hist))
                lbp_energy = np.sum(lbp_prob**2)
                lbp_entropy = -np.sum(np.multiply(lbp_prob, np.log2(lbp_prob)))

                # File name without .jpg extension
                name = os.path.splitext(filename)[0]

                # Escribir los resultados en el archivo .csv
                writer.writerow([name, contrast, dissimilarity, homogeneity, ASM, energy, correlation, entropy, 
                                 gabor, gabor_0, gabor_45, gabor_90, gabor_135, hog_features_mean, lbp_entropy, category])

print('Finish')

c:\Users\pepe-\anaconda3\Lib\site-packages\skimage\feature\texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


Finish
